## Imports

In [108]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import pandas as pd
from geojson import Feature, FeatureCollection, Point
from datetime import datetime, timedelta
import re
import os

## PD Set Options

In [109]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 1600)

## Get desired date variables

In [110]:
week_start = '2022-6-11'
week_end = '2022-12-11'

week_start = datetime.strptime(week_start, '%Y-%d-%m')
week_end = datetime.strptime(week_end, '%Y-%d-%m')

## Read-in Data

In [111]:
df = pd.read_csv('redfin_2022-11-15-14-49-21.csv', dtype={'YEAR BUILT':'str'})

In [112]:
df['PRICE_INT'] = df['PRICE']

In [113]:
len(df)

228

In [114]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME',
       'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
       'SOURCE', 'MLS#', 'FAVORITE', 'INTERESTED', 'LATITUDE', 'LONGITUDE',
       'PRICE_INT'],
      dtype='object')

In [115]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [116]:
df = df[(df['SOLD DATE'] >= week_start) & (df['SOLD DATE'] <= week_end)]

In [117]:
len(df)

36

In [118]:
df = df.sort_values(by='PRICE', ascending=False).head(10)
df['PRICE'] = df['PRICE'].apply(lambda x: '${:,}'.format(x))
df['$/SQUARE FEET'] = df['$/SQUARE FEET'].apply(lambda x: '${:,}'.format(x))

## Correction Section

## Map making section

In [119]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME',
       'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
       'SOURCE', 'MLS#', 'FAVORITE', 'INTERESTED', 'LATITUDE', 'LONGITUDE',
       'PRICE_INT'],
      dtype='object')

In [120]:
df

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,PRICE_INT
137,PAST SALE,2022-11-10,Condo/Co-op,2550 N Lakeview Ave Unit T3501,Chicago,IL,60614,"$5,825,000",4.0,4.5,Lincoln Park 2550,4700.0,NaN,2010,NaN,"$1,239.0",6925.0,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/2550-N-Lakeview-Ave-60614/unit-T3501/home/182228665,MRED,11648944.0,N,Y,41.929151,-87.640113,5825000
152,PAST SALE,2022-11-07,Condo/Co-op,11 E Walton Pl #4002,Chicago,IL,60611,"$5,330,000",3.0,3.5,CHI - Near North Side,4000.0,NaN,2009,NaN,"$1,333.0",3946.0,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/11-E-Walton-St-60611/unit-4002/home/40378386,MRED,11449266.0,N,Y,41.899701,-87.627604,5330000
31,PAST SALE,2022-11-08,Condo/Co-op,211 N Harbor Dr #4501,Chicago,IL,60601,"$3,442,850",4.0,3.5,CHI - Loop,3002.0,NaN,2021,NaN,"$1,147.0",2516.0,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/211-N-Harbor-Dr-60601/unit-4501/home/176797249,MRED,11250927.0,N,Y,41.885638,-87.615471,3442850
151,PAST SALE,2022-11-07,Condo/Co-op,1625 N Burling St #402,Chicago,IL,60614,"$2,360,000",3.0,2.5,CHI - Lincoln Park,NaN,NaN,2017,NaN,$nan,1176.0,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/1625-N-Burling-St-60614/unit-402/home/167585986,MRED,11469143.0,N,Y,41.911641,-87.646919,2360000
70,PAST SALE,2022-11-08,Condo/Co-op,363 E Wacker Dr #3509,Chicago,IL,60601,"$2,300,000",3.0,3.5,CHI - Loop,2846.0,NaN,2021,NaN,$808.0,1150.0,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/363-E-Wacker-Dr-60601/unit-3509/home/104321911,MRED,11450286.0,N,Y,41.887457,-87.618373,2300000
103,PAST SALE,2022-11-08,Condo/Co-op,323 S Racine Ave Unit PHC,Chicago,IL,60607,"$1,755,900",4.0,3.5,CHI - Near West Side,3103.0,NaN,2021,NaN,$566.0,601.0,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/323-S-Racine-Ave-60607/unit-PHC/home/174119889,MRED,10964220.0,N,Y,41.877432,-87.656836,1755900
97,PAST SALE,2022-11-10,Condo/Co-op,676 N Kingsbury St #204,Chicago,IL,60654,"$1,610,000",4.0,4.5,CHI - Near North Side,3081.0,NaN,2016,NaN,$523.0,1859.0,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/676-N-Kingsbury-St-60654/unit-204/home/171319841,MRED,11296816.0,N,Y,41.894490,-87.641600,1610000
76,PAST SALE,2022-11-11,Condo/Co-op,363 E Wacker Dr #1502,Chicago,IL,60601,"$1,575,000",3.0,2.0,CHI - Loop,2292.0,NaN,2021,NaN,$687.0,970.0,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/363-E-Wacker-Dr-60601/unit-1502/home/178672331,MRED,11432192.0,N,Y,41.887583,-87.617310,1575000
136,PAST SALE,2022-11-10,Condo/Co-op,540 W Webster Ave Unit 1012-13,Chicago,IL,60614,"$1,560,000",3.0,3.5,CHI - Lincoln Park,2596.0,NaN,2017,NaN,$601.0,1840.0,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/540-W-Webster-Ave-60614/unit-1012-13/home/144402983,MRED,11609249.0,N,Y,41.922048,-87.643023,1560000
62,PAST SALE,2022-11-07,Condo/Co-op,2350 N Orchard St Unit MZ102,Chicago,IL,60614,"$1,500,000",2.0,2.5,CHI - Lincoln Park,2017.0,NaN,2020,NaN,$744.0,1343.0,Sold,NaN,NaN,https://www.redfin.com/IL/Chicago/2350-N-Orchard-St-60614/unit-MZ102/home/180784094,MRED,11481091.0,N,Y,41.925102,-87.646499,1500000


In [121]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(), zoom_start=12)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Top 10 Cook County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('November 6th - November 12th, as per the MLS')

for i,r in df.iterrows():
    location = (r["LATITUDE"], r["LONGITUDE"])
    info = (r["PRICE"], r['$/SQUARE FEET'], r['ADDRESS'], r['YEAR BUILT'], r['BEDS'], r['BATHS'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Address: {bold_start}{info[2]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}\
    Price sq ft: {bold_start}{info[1]}{bold_end}{new_line} \
    Year Built: {bold_start}{info[-3]}{bold_end}{new_line}\
    Beds: {bold_start}{info[-2]}{bold_end}{new_line}\
    Baths: {bold_start}{info[-1]}{bold_end}{new_line}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [122]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [123]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [124]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/chi_weekly_condo_sales_11_7_11_11_2022


In [133]:
print(f"Mean ${df['PRICE_INT'].mean():,}")
print('---------')
print(f"Median ${df['PRICE_INT'].median():,}")
print('---------')
print(f"Sum ${df['PRICE_INT'].sum():,}")
print('---------')
print(f"Sum square feet {df['SQUARE FEET'].sum():,}")

Mean $2,725,875.0
---------
Median $2,027,950.0
---------
Sum $27,258,750
---------
Sum square feet 27,637.0


In [135]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME',
       'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
       'SOURCE', 'MLS#', 'FAVORITE', 'INTERESTED', 'LATITUDE', 'LONGITUDE',
       'PRICE_INT'],
      dtype='object')

In [144]:
df.filter(items=['SOLD DATE','LOCATION','ADDRESS','PRICE','BEDS','BATHS','SQUARE FEET','YEAR BUILT'])

,SOLD DATE,LOCATION,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT
137,2022-11-10,Lincoln Park 2550,2550 N Lakeview Ave Unit T3501,"$5,825,000",4.0,4.5,4700.0,2010
152,2022-11-07,CHI - Near North Side,11 E Walton Pl #4002,"$5,330,000",3.0,3.5,4000.0,2009
31,2022-11-08,CHI - Loop,211 N Harbor Dr #4501,"$3,442,850",4.0,3.5,3002.0,2021
151,2022-11-07,CHI - Lincoln Park,1625 N Burling St #402,"$2,360,000",3.0,2.5,NaN,2017
70,2022-11-08,CHI - Loop,363 E Wacker Dr #3509,"$2,300,000",3.0,3.5,2846.0,2021
103,2022-11-08,CHI - Near West Side,323 S Racine Ave Unit PHC,"$1,755,900",4.0,3.5,3103.0,2021
97,2022-11-10,CHI - Near North Side,676 N Kingsbury St #204,"$1,610,000",4.0,4.5,3081.0,2016
76,2022-11-11,CHI - Loop,363 E Wacker Dr #1502,"$1,575,000",3.0,2.0,2292.0,2021
136,2022-11-10,CHI - Lincoln Park,540 W Webster Ave Unit 1012-13,"$1,560,000",3.0,3.5,2596.0,2017
62,2022-11-07,CHI - Lincoln Park,2350 N Orchard St Unit MZ102,"$1,500,000",2.0,2.5,2017.0,2020
